# Browse experiment results ...

In [9]:
import torch
import pickle
import glob
import os.path
import pandas as pd
import core.train_engine

from core.utils_ipynb import read_exp_result_files


def get_keychain_value_iter(d, key_chain=None):
    key_chain = [] if key_chain is None else list(key_chain).copy()       
    
    if not isinstance(d, dict):
        
        yield tuple(key_chain), d
    else:
        for k, v in d.items():
            yield from get_keychain_value_iter(v, key_chain + [k])
            
def get_keychain_value(d, key_chain):
    
    try:
        for k in key_chain:
            d = d[k]
            
    except Exception as ex:
        raise KeyError() from ex
        
    return d

In [11]:
kc = {k: k[-1] for k, v in list(get_keychain_value_iter(core.train_engine.__exp_res_meta))}
kc

{('exp_cfg', 'dataset_name'): 'dataset_name',
 ('exp_cfg', 'training', 'lr'): 'lr',
 ('exp_cfg', 'training', 'lr_drop_fact'): 'lr_drop_fact',
 ('exp_cfg', 'training', 'num_epochs'): 'num_epochs',
 ('exp_cfg', 'training', 'epoch_step'): 'epoch_step',
 ('exp_cfg', 'training', 'batch_size'): 'batch_size',
 ('exp_cfg', 'training', 'weight_decay'): 'weight_decay',
 ('exp_cfg', 'training', 'validation_ratio'): 'validation_ratio',
 ('exp_cfg', 'model', 'model_type'): 'model_type',
 ('exp_cfg',
  'model',
  'use_super_level_set_filtration'): 'use_super_level_set_filtration',
 ('exp_cfg', 'model', 'use_node_degree'): 'use_node_degree',
 ('exp_cfg',
  'model',
  'set_node_degree_uninformative'): 'set_node_degree_uninformative',
 ('exp_cfg', 'model', 'pooling_strategy'): 'pooling_strategy',
 ('exp_cfg', 'model', 'use_node_label'): 'use_node_label',
 ('exp_cfg', 'model', 'gin_number'): 'gin_number',
 ('exp_cfg', 'model', 'gin_dimension'): 'gin_dimension',
 ('exp_cfg', 'model', 'gin_mlp_type'): 'gi

In [12]:
COL_NAMES = {
    ('exp_cfg', 'dataset_name'): 'dataset_name',
    #('exp_cfg', 'tag'): 'tag', 
#     ('exp_cfg', 'training', 'lr'): 'lr',
#     ('exp_cfg', 'training', 'lr_drop_fact'): 'lr_drop_fact',
#     ('exp_cfg', 'training', 'num_epochs'): 'num_epochs',
#     ('exp_cfg', 'training', 'epoch_step'): 'epoch_step',
    ('exp_cfg', 'training', 'batch_size'): 'batch_size',
#     ('exp_cfg', 'training', 'weight_decay'): 'weight_decay',
#     ('exp_cfg', 'training', 'validation_ratio'): 'validation_ratio',
    ('exp_cfg', 'model', 'model_type'): 'model_type',
    ('exp_cfg', 'model', 'use_super_level_set_filtration'): 'use_super_level_set_filtration',
    ('exp_cfg', 'model', 'use_node_degree'): 'use_node_degree',
    ('exp_cfg', 'model', 'use_node_label'): 'use_node_label',
    ('exp_cfg', 'model', 'gin_number'): 'gin_number',
    ('exp_cfg', 'model', 'gin_dimension'): 'gin_dimension',
    #('exp_cfg', 'model', 'gin_mlp_type'): 'gin_mlp_type',
    ('exp_cfg', 'model', 'set_node_degree_uninformative'): 'set_node_degree_uninformative',
    ('exp_cfg', 'model', 'num_struct_elements'): 'num_struct_elements',
    ('exp_cfg', 'model', 'drop_out'): 'drop_out',
    ('exp_cfg', 'model', 'pooling_strategy'): 'pooling_strategy',
#     ('cv_test_acc',): 'cv_test_acc',
#     ('cv_val_acc',): 'cv_val_acc',
#     ('cv_indices_trn_tst_val',): 'cv_indices_trn_tst_val',
#     ('cv_epoch_loss',): 'cv_epoch_loss',
#     ('start_time',): 'start_time',
#     ('id',): 'id',
    ('finished_training',): 'finished_training'
}

In [13]:
def pd_frame(path):
    
    f = read_exp_result_files(path)
    
    data_frames = []
    for i, res in enumerate(f):
        row = {}
        
        cv_acc_last = [x[-1] for x in res['cv_test_acc'] if len(x) > 0]
        
        row['acc_last_mean'] = np.mean(cv_acc_last)
        row['acc_last_std'] = np.std(cv_acc_last)
        
        cv_acc_validated = []
        for test, val in zip(res['cv_test_acc'], res['cv_val_acc']):
            if not len(test) == res['exp_cfg']['training']['num_epochs']:
                continue
            n = len(test)//2
            test = torch.tensor(test[n:])
            val = torch.tensor(val[n:])
            #test = torch.tensor(test)
            #val = torch.tensor(val)
            
            
            _, i_max = val.max(0)
            cv_acc_validated.append(test[i_max].item())
            
        row['acc_val_mean'] = np.mean(cv_acc_validated)
        row['acc_val_std'] = np.std(cv_acc_validated)
        
        
        cv_folds_available = sum([1 for cv in res['cv_test_acc'] if len(cv) == res['exp_cfg']['training']['num_epochs']])
        row['cv_folds_available'] = cv_folds_available
        
        
        for k, v in COL_NAMES.items():
            try:
                row[v] = get_keychain_value(res, k)
            except KeyError:
                pass

        f = pd.DataFrame(row, index=[i])
        
        data_frames.append(f)
        
        
    return pd.concat(data_frames, sort=True)

In [63]:
path = './results/'
RES = pd_frame(path)
RES

,acc_last_mean,acc_last_std,acc_val_mean,acc_val_std,batch_size,cv_folds_available,dataset_name,drop_out,finished_training,gin_dimension,gin_number,model_type,num_struct_elements,pooling_strategy,set_node_degree_uninformative,use_node_degree,use_node_label,use_super_level_set_filtration
0,73.313224,1.433772,72.771075,2.718653,64,10,PROTEINS,0.0,True,64.0,1.0,PershomLearnedFilt,100.0,NaN,False,True,False,True
1,50.066667,3.217314,50.666667,2.616189,64,10,IMDB-MULTI,0.5,True,64.0,1.0,GIN,NaN,NaN,False,True,False,NaN
2,49.333333,4.381780,50.600000,3.032418,64,10,IMDB-MULTI,0.5,True,64.0,5.0,GIN,NaN,NaN,False,True,False,NaN
3,72.600000,3.231099,73.600000,3.261901,64,10,IMDB-BINARY,0.5,True,64.0,1.0,GIN,NaN,NaN,False,True,False,NaN
4,74.900000,4.437342,73.100000,3.176476,64,10,IMDB-BINARY,0.5,True,64.0,5.0,GIN,NaN,NaN,False,True,False,NaN
5,81.838487,1.640846,82.058849,1.731814,64,10,COLLAB,0.5,True,64.0,1.0,GIN,NaN,NaN,False,True,False,NaN
6,82.319769,1.669858,82.139409,1.894722,64,10,COLLAB,0.5,True,64.0,5.0,GIN,NaN,NaN,False,True,False,NaN
7,38.333333,6.146363,37.166667,4.657730,64,10,ENZYMES,0.5,True,64.0,1.0,GIN,NaN,NaN,False,True,True,NaN
8,36.333333,3.711843,34.333333,6.591240,64,10,ENZYMES,0.5,True,64.0,5.0,GIN,NaN,NaN,False,True,True,NaN
9,30.833333,6.465722,30.333333,6.658328,64,10,ENZYMES,0.5,True,64.0,1.0,GIN,NaN,NaN,False,False,True,NaN


The following cells contain some utility for messing around with results, i.e., deleting etc. 